In [11]:
%%time
#installing required packages
import pandas as pd
import numpy as np
import psycopg2
import time
from IPython import get_ipython
from datetime import date, datetime, timedelta
from dateutil.relativedelta import relativedelta
import os
import sys
import io
from io import StringIO
import boto3
from boto3 import client
import smtplib,ssl
from email.mime.multipart import MIMEMultipart
from email.mime.base import MIMEBase
from email.mime.text import MIMEText
from email.utils import COMMASPACE
from email import encoders

#project_path = os.path.abspath(os.path.join(os.path.join( os.path.dirname( __file__ ), os.path.pardir ),os.path.pardir))

def get_postgres_connection():
    # Establish connection to postgres parameter
    try:
        connection = psycopg2.connect(dbname= 'projectxmultitenant', 
                                      host='omni-dms-shipsy-readreplica.cwehsc8qlrqj.us-west-2.rds.amazonaws.com',
                                      user= '    ', 
                                      password= '    ')

        print("Connection to shipsy is Successfull")
        return connection

    except Exception as er:
        print("Exception ", er)
    

engine=None
    
def query():
    q1='''select  
left(b.worker_code,4)::int as "Store Code",
case when left(b.worker_code,4)='3089' then 'Brookefield'
when left(b.worker_code,4)='3093' then 'HSR Layout' 
when left(b.worker_code,4)='1111' then 'Mathikere'
when left(b.worker_code,4)='1135' then 'CIL Layout'
when left(b.worker_code,4)='1699' then 'KORAMANGALA - 2'
when left(b.worker_code,4)='3015' then 'V V Puram Bangalore'
when left(b.worker_code,4)='3219' then 'R R Nagar BEML Layout Bangalore'
when left(b.worker_code,4)='3233' then 'Hosakerehalli Bangalore'
when left(b.worker_code,4)='3248' then 'Banashankari 2nd Stage Bangalore'
when left(b.worker_code,4)='3313' then 'MEI layout Bangalore'
when left(b.worker_code,4)='3412' then 'Varthur Road Bangalore'
when left(b.worker_code,4)='3512' then 'Rajajinagar 2nd Block'
when left(b.worker_code,4)='3559' then 'Bagalur SKN'
when left(b.worker_code,4)='3195' then 'Subbanna Garden'
end as "Store Name",
b.worker_code::int as "Worker Code", 
b.name "Worker Name",
a.workdate::date as "Date",
(min(case when (extract(epoch from a.offduty_time - a.onduty_time)/60)::decimal(8,2) >60 then onduty_time end)::timestamp) + (interval '5 hour')+(interval '30 minute') as "On Duty Time", 
(max(case when (extract(epoch from a.offduty_time - a.onduty_time)/60)::decimal(8,2) >60 then least(offduty_time, (a.workdate+1)::timestamp) end)::timestamp) + (interval '5 hour')+(interval '30 minute') as "Off Duty Time", 
(extract(epoch from max(case when (extract(epoch from a.offduty_time - a.onduty_time)/60)::decimal(8,2) >60 then least(offduty_time, (a.workdate+1)::timestamp) end)::timestamp - 
min(case when (extract(epoch from a.offduty_time - a.onduty_time)/60)::decimal(8,2) >60 then onduty_time end)::timestamp )/3600)::decimal(8,2) as "Login Hours",
max(coalesce(c.orders_delivered,0)) as "Delivered Count",
max(coalesce(c.km_travelled,0)) as "Distance Travelled (KM)",
max(coalesce(c.weight,0)) as "Order Weight (Kg)"
from public.workeronduty a
inner join public.worker b on a.worker_id =b.id
left join (select distinct a.worker_id, a.updated_at::date, count(distinct a.consignment_id) as orders_delivered ,
sum(nullif(split_part(split_part(b.extra_details::text,'"expected_distance": ',2),',',1),'')::float/1000.0) as km_travelled,
sum(nullif(split_part(split_part(b.extra_details::text,'"order_size": ',2),',',1),'')::float) as weight
from public.task a 
left join public.consignment b on a.consignment_id=b.id
where task_type='delivery'
group by 1,2) c on a.worker_id=c.worker_id and a.workdate::date=c.updated_at::date
where workdate::date = current_date-1
group by 1,2,3,4,5 order by 1
'''
    
    q2='''select *,case when d.delivered_time>d.slot_end_time then 'Y' else 'N' end as breach from 

(select  
left(b.worker_code,4)::int as "Store Code",
case when left(b.worker_code,4)='3089' then 'Brookefield'
when left(b.worker_code,4)='3093' then 'HSR Layout' 
when left(b.worker_code,4)='1111' then 'Mathikere'
when left(b.worker_code,4)='1135' then 'CIL Layout'
when left(b.worker_code,4)='1699' then 'KORAMANGALA - 2'
when left(b.worker_code,4)='3015' then 'V V Puram Bangalore'
when left(b.worker_code,4)='3219' then 'R R Nagar BEML Layout Bangalore'
when left(b.worker_code,4)='3233' then 'Hosakerehalli Bangalore'
when left(b.worker_code,4)='3248' then 'Banashankari 2nd Stage Bangalore'
when left(b.worker_code,4)='3313' then 'MEI layout Bangalore'
when left(b.worker_code,4)='3412' then 'Varthur Road Bangalore'
when left(b.worker_code,4)='3512' then 'Rajajinagar 2nd Block'
when left(b.worker_code,4)='3559' then 'Bagalur SKN'
when left(b.worker_code,4)='3195' then 'Subbanna Garden'
end as "Store Name",
b.worker_code::int as "Worker Code", 
b.name "Worker Name",
a.workdate::date as "Date",
(min(case when (extract(epoch from a.offduty_time - a.onduty_time)/60)::decimal(8,2) >60 then onduty_time end)::timestamp)+ (interval '5 hour')+(interval '30 minute') as "On Duty Time", 
(max(case when (extract(epoch from a.offduty_time - a.onduty_time)/60)::decimal(8,2) >60 then least(offduty_time, (a.workdate+1)::timestamp) end)::timestamp)+ (interval '5 hour')+(interval '30 minute') as "Off Duty Time", 
(extract(epoch from max(case when (extract(epoch from a.offduty_time - a.onduty_time)/60)::decimal(8,2) >60 then least(offduty_time, (a.workdate+1)::timestamp) end)::timestamp - 
min(case when (extract(epoch from a.offduty_time - a.onduty_time)/60)::decimal(8,2) >60 then onduty_time end)::timestamp )/3600)::decimal(8,2) as "Login Hours",
--c.orders_delivered as "Order delivered Consignment ID",
order_no,
(to_timestamp(delivered_epoch_time::bigint/1000)::timestamp)+ (interval '5 hour')+(interval '30 minute') as delivered_time,
(to_timestamp(slot_start_epoch_time::bigint/1000)::timestamp)+ (interval '5 hour')+(interval '30 minute') as slot_start_time,
(to_timestamp(slot_end_epoch_time::bigint/1000)::timestamp)+ (interval '5 hour')+(interval '30 minute') as slot_end_time,
max(coalesce(c.km_travelled,0)) as "Distance Travelled (KM)",
max(coalesce(c.weight,0)) as "Order Weight (Kg)"
from public.workeronduty a
inner join public.worker b on a.worker_id =b.id
left join (select distinct a.worker_id, a.updated_at::date,  a.consignment_id as orders_delivered ,b.reference_number as order_no,
nullif(left(replace(replace(split_part(b.extra_details::text, 'delivered_time',2),'"',''),': ',''),13),'') as delivered_epoch_time,
nullif(left(replace(replace(split_part(b.extra_details::text, 'delivery_time_slot_start',2),'"',''),': ',''),13),'') as slot_start_epoch_time,
nullif(left(replace(replace(split_part(b.extra_details::text, 'delivery_time_slot_end',2),'"',''),': ',''),13),'') as slot_end_epoch_time,
sum(nullif(split_part(split_part(b.extra_details::text,'"expected_distance": ',2),',',1),'')::float/1000.0) as km_travelled,
sum(nullif(split_part(split_part(b.extra_details::text,'"order_size": ',2),',',1),'')::float) as weight
from public.task a 
left join public.consignment b on a.consignment_id=b.id
where task_type='delivery'
group by 1,2,3,4,5,6,7) c on a.worker_id=c.worker_id and a.workdate::date=c.updated_at::date
where workdate::date = current_date-1
group by 1,2,3,4,5,9,10,11,12 order by 1
)d

'''
    
    return q1,q2

def data_preprocess(conn, query, last_record_date):
    # Get records from postgres shipsy Server
    print("Fetching records from DB")
    start = time.time()
    result=pd.read_sql(query.replace('last_record_date',last_record_date),conn)
    end = time.time()

    print("Time for fetching {} records is {} seconds".format(result.shape[0], end - start))
    return result 

#Testing
import smtplib,ssl
from email.mime.multipart import MIMEMultipart
from email.mime.base import MIMEBase
from email.mime.text import MIMEText
from email.utils import COMMASPACE
from email import encoders
import os
from pretty_html_table import build_table
#import html
#html = "<h1>TESTING EMAIL</h1>"


def send_mail(summary,send_from, send_to, send_to_cc, subject, content, files, server="localhost"):
    assert type(send_to)==list
    assert type(files)==list
    body = """Hi Team,<br><br>PFA the DE attendance report for yesterday.<br><br> Kindly reach out to us in case of any concerns
              
              <br><br><br>
              Thanks,<br>
              Ratnadip Bhorge<br> 
              www.ganitinc.com"""
    
    #view=build_table(summary, 'blue_light',width='50px',font_size='medium',text_align='center')
    #body+=view
    
    #body+="<br><br>Kindly reach out to us in case of any concerns.<br><br>"
    msg = MIMEMultipart()
    part1 = MIMEText(body, 'html')
    msg.attach(part1)
    msg['From'] = send_from
    msg['To'] = ', '.join(send_to)
    msg['Cc'] = ', '.join(send_to_cc)
    msg['Subject'] = subject
    

    for f in files:
        print(f)
        part = MIMEBase('application', "octet-stream")
        part.set_payload( open(f,"rb").read() )
        encoders.encode_base64(part)
        part.add_header('Content-Disposition', 'attachment; filename="%s"' % os.path.basename(f))
        msg.attach(part)
    context=ssl.create_default_context()
    with smtplib.SMTP("smtp.office365.com", port=587) as smtp:
        smtp.starttls(context=context)
        smtp.login(msg["From"], "password" )
        smtp.send_message(msg)


list_to = ['ratnadip.bhorge@ganitinc']
list_cc = ['ashutosh@ganitinc.com']
#list_cc = ['sayoni.chowdhury@ganitinc.com']
#list_cc = ['ashutosh@ganitinc.com']


def formatting(summary,order):
        x=date.today()
        
        writer=pd.ExcelWriter(f'C:\\Users\\Ratnadip.bhorge\\Documents\\Python Scripts\\DE Attendance Report.xlsx',engine='xlsxwriter',options={'nan_inf_to_errors': True})
        
        summary.to_excel(writer, index=False, sheet_name='summary')
        order.to_excel(writer, index=False, sheet_name='orderleveldetails')
        #slc.to_excel(writer, index=False, sheet_name='SLC_Recommended_Vendor_Data')

        workbook = writer.book
        worksheet_1 = writer.sheets['summary']
        worksheet_2 = writer.sheets['orderleveldetails']
        #worksheet_3 = writer.sheets['SLC_Recommended_Vendor_Data']
        
         # Add a header format.
        header_format = workbook.add_format({
            'bold': True,
            'align': 'center',
            'valign': 'vcenter',
            'fg_color': '#DCE6F1',
            'border': 1})

        for col_num, value in enumerate(summary.columns.values):
            worksheet_1.write(0, col_num, value, header_format)

            column_len = summary[value].astype(str).str.len().max()
            # Setting the length if the column header is larger
            # than the max column value length
            column_len = max(column_len, len(value)) + 1
            #print(column_len)
            # set the column length
            worksheet_1.set_column(col_num, col_num, column_len)
        
        for col_num, value in enumerate(order.columns.values):
            worksheet_2.write(0, col_num, value, header_format)

            column_len = order[value].astype(str).str.len().max()
            # Setting the length if the column header is larger
            # than the max column value length
            column_len = max(column_len, len(value)) + 1
            #print(column_len)
            # set the column length
            worksheet_2.set_column(col_num, col_num, column_len)



        writer.save()
        summary=summary
        send_mail(summary,'ratnadip.bhorge@ganitinc.com', list_to,list_cc, "Daily DE Attendance Report ","Daily DE Attendance Report", [f'C:\\Users\\Ratnadip.bhorge\\Documents\\Python Scripts\\DE Attendance Report.xlsx'])


def main_function():
    
    last_record_date = date.today()
    

    # Get the redshift Connection Object
    connection=get_postgres_connection()

    # Query for fetching records
    query1,query2 = query()

    # Preprocessing the records
    preprocessed_result1 = data_preprocess(connection, query1,str(last_record_date))
    preprocessed_result2 = data_preprocess(connection, query2,str(last_record_date))
    

    # Get pivotted data 
    recom_data=preprocessed_result1
    order=preprocessed_result2
    
    summary=recom_data
    #Creating excel files 
    formatting(summary,order)


if __name__ == "__main__":
    main_function()   

Connection to shipsy is Successfull
Fetching records from Redshift
Time for fetching 21 records is 0.5238406658172607 seconds
Fetching records from Redshift
Time for fetching 46 records is 0.2823469638824463 seconds
C:\Users\Ratnadip.bhorge\Documents\Python Scripts\DE Attendance Report.xlsx


<timed exec>:199: FutureWarning: Use of **kwargs is deprecated, use engine_kwargs instead.


Wall time: 3.82 s
